## MNILST MLP

In [1]:
!pip install matplotlib
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 39.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 4.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pillow
    Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0


In [2]:
# Library
import torch
import numpy as np
import matplotlib as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [4]:
# device 설정
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print(DEVICE)

cuda


In [5]:
# Hyperparameter setting
batch_size = 32
epochs = 10

In [8]:
# MNIST DATA LOAD
train_dataset = datasets.MNIST(root="../data/MNIST", train = True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="../data/MNIST", train = False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle = True)

In [9]:
# Dataset 확인
for (X_train, y_train) in train_loader:
    print('X_train size:',X_train.size(), 'type:', X_train.type())
    print('y_train size:',y_train.size(), 'type:', y_train.type())
    break

X_train size: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train size: torch.Size([32]) type: torch.LongTensor


In [14]:
# MLP 모듈 설계

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28 , 512)
        self.fc2 = nn.Linear(512 , 256)
        self.fc3 = nn.Linear(256 , 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28) # flatten 느낌
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [15]:
# optimizer, obj function 설정
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)
criterian = nn.CrossEntropyLoss()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [16]:
# 학습 데이터에 대한 model 성능 확인 함수 정의
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterian(output,label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train_epoch : {} [{}/{}({:.0f}%)] \tTrain Loss : {:.6f}".format(epochs, batch_idx*len(image),len(train_loader.dataset), 100*batch_idx/len(train_loader), loss.item()))

In [19]:
# 검증 데이터에 대한 모델 성능 확인하는 함수 정의
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterian(output,label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct +=prediction.eq(label.view_as(prediction)).sum().item() 

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [20]:
# Train, Test 진행
for epoch in range(1, epochs+1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[Epoch: {}], \tTest loss: {:.4f}, \tTest acc : {:.2f}".format(epoch, test_loss, test_accuracy))

/home/seonggye/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train_epoch : 10 [0/60000(0%)] 	Train Loss : 2.259018
Train_epoch : 10 [6400/60000(11%)] 	Train Loss : 2.327599
Train_epoch : 10 [12800/60000(21%)] 	Train Loss : 2.296334
Train_epoch : 10 [19200/60000(32%)] 	Train Loss : 2.357299
Train_epoch : 10 [25600/60000(43%)] 	Train Loss : 2.290432
Train_epoch : 10 [32000/60000(53%)] 	Train Loss : 2.335135
Train_epoch : 10 [38400/60000(64%)] 	Train Loss : 2.307302
Train_epoch : 10 [44800/60000(75%)] 	Train Loss : 2.300529
Train_epoch : 10 [51200/60000(85%)] 	Train Loss : 2.251631
Train_epoch : 10 [57600/60000(96%)] 	Train Loss : 2.217779

[Epoch: 1], 	Test loss: 0.0694, 	Test acc : 26.63
Train_epoch : 10 [0/60000(0%)] 	Train Loss : 2.196549
Train_epoch : 10 [6400/60000(11%)] 	Train Loss : 2.166552
Train_epoch : 10 [12800/60000(21%)] 	Train Loss : 2.109120
Train_epoch : 10 [19200/60000(32%)] 	Train Loss : 2.026603
Train_epoch : 10 [25600/60000(43%)] 	Train Loss : 2.056266
Train_epoch : 10 [32000/60000(53%)] 	Train Loss : 1.880376
Train_epoch : 10 